In [ ]:
#FILE CON I MIEI TENTATIVI DI PREPROCESSING SUL DATABASE DI ANOBii
from pyspark.sql.functions import lower
from pyspark.sql.types import IntegerType, StringType
file = "/data/SMARTDATA/books/anobii/author_item.csv"
DFauthorbooks = spark.read.csv(file,header=True) #MOSTRA I LIBRI RELATIVI AGLI AUTORI
#DFauthorbooks.show(1, False)

#LIBRI DEGLI AUTORI

In [ ]:
file = "/data/SMARTDATA/books/anobii/author_display.csv"
DFdisplay = spark.read.csv(file,header=True) #MOSTRA GLI AUTORI DEL DATABASE
#DFdisplay.show()
#DFauthorbooks_withnames = DFauthorbooks.join(DFdisplay, DFauthorbooks.author_id == DFdisplay.author_id).drop(DFdisplay.author_id)
#AUTORI

In [ ]:
#A quanto pare author_display è una tabella contenente tutti gli autori del database di anobii mentre author_items
#contiene i diversi libri associati ad ogni autore. 
#DUBBIO: cosa è author original? Cerchiamo una tupla in cui author_original NON è NULL:

file = "/data/SMARTDATA/books/anobii/item.csv"
DFitems_anobii = spark.read.csv(file,header=True) #MOSTRA I LIBRI DEL DATABASE
#DFitems.show()

#DFitemsauthor = DFitems_anobii.join(DFauthorbooks_withnames, DFitems_anobii.item_id == DFauthorbooks.item_id).drop(DFitems_anobii.item_id)
#DFitemsauthor.filter(DFitemsauthor.author_id_original != "null").show(20, False)
#DFitemsauthor.show(1, False)
#AUTHOR: JOHN FOWLES
#DFdisplay.filter(DFdisplay.author_id == "360704").show(20, False)
#DFdisplay.filter(DFdisplay.author_id_original == "72565").show(20, False)

#IPOTESI: Sembra che author_id_original sia semplicemente l'id di un vecchio database non a nostra disposizione
#dato che non è un autore (author_id) presente in questo DB

#Possiamo anche notare come language sia il linguaggio del libro mentre language_correct sia il linguaggio del
#libro originale (notare la tupla item_id 1002883)

In [ ]:
file = "/data/SMARTDATA/books/anobii/language_mapping.csv"
DFlanguages = spark.read.csv(file,header=True)
#DFlanguages.show()

#Dobbiamo tenere solo i libri con language=11 #FILTRO LINGUAGGIO



In [ ]:
file = "/data/SMARTDATA/books/anobii/item_recommend.csv"
DFrecommends = spark.read.csv(file,header=True)
#DFrecommends.show(50, False)

#Questa tabella contiene gli ISBN riferiti all'item Id (quindi item_id non è unico?)

In [ ]:
#Per coerenza, procediamo a un filtraggio equivalente a quello fatto per le biblioteche:
#FILTRAGGI BASE

#FILTRAGGIO 1: solo libri italiani (filtrare language==11)
#FILTRAGGIO 2: solo monografie e manoscritti (questo a dire il vero è fattibile in maniera un po' diversa dal caso
#delle biblioteche poiché l'attributo "binding" indica la rilegatura del libro). Per iniziare, considererò
#"paperback" (copertina flessibile) e "hardcover" (copertina rigida) NOTA: probabilmente ulteriori scremature
#da questo punto di vista saranno possibili
#FILTRAGGIO 3: eliminare fumetti e magazine (non sono sicuro della loro presenza, controllo)
#UPDATE: non solo ci sono ma sono pure tantissimi! Filtraggio doveroso

#DFitems.filter(DFitems.title.contains("Topolino")).show(20, False)

#FILTRAGGI AVANZATI

#FILTRAGGIO 4: Libri con più di 50 voti (Dove prendere i voti?) In item_comment_vote ci sono i commenti ai libri
#fatti da vari utenti. Tuttavia, non vedo i voti. item_comment_feedback contiene i commenti veri e propri.
#In item_comment_vote sembrano esserci due campi "total_yes" e "total_no": che siano loro i voti? (Consigliato
#Non consigliato). UPDATE: Noto che in item_comment_vote è presente l'average rating: è già qualcosa, tuttavia
#vorrei trovare i voti dei singoli utenti.

#FILTRAGGIO 5: Pulizie eventuali, su anobii i titoli sembrano già puliti quindi è necessario semplicemente mappare
#l'id dell'autore con il nome dell'autore.

file = "/data/SMARTDATA/books/anobii/item_comment_vote.csv" #MOSTRA INFORMAZIONI SUL VOTO DEGLI UTENTI
DFvotes = spark.read.csv(file,header=True)
#DFvotes.show(50, False)

file = "/data/SMARTDATA/books/anobii/item_comment_feedback.csv" #MOSTRA IL VOTO MEDIO
DFfeedback = spark.read.csv(file,header=True)
#DFfeedback.show(50, False)

file = "/data/SMARTDATA/books/anobii/person_item_recommend.csv" #MOSTRA I COMMENTI DEGLI UTENTI RELATIVI A UN LIBRO
DFcomments = spark.read.csv(file,header=True)
#DFcomments.show(50, False)

file = "/data/SMARTDATA/books/anobii/person_item_progress.csv" #MOSTRA INFORMAZIONI SULLE RACCOMANDAZIONI
DFprogress = spark.read.csv(file,header=True)
#DFprogress.show(50, False)

file = "/data/SMARTDATA/books/anobii/link_person_item_comment.csv" #MOSTRA INFORMAZIONI VARIE QUALI LA PRESENZA DI
#SPOILER IN UN COMMENTO O LA PRESENZA DI PROFANITà
DFinfo = spark.read.csv(file,header=True)
#DFinfo.show(50, False)

In [14]:
#INIZIAMO A FILTRARE USANDO LA ROADMAP DELLA CELLA POCO SOPRA

#FILTRAGGIO 1: linguaggio italiano
#Mi basterà filtrare da item.csv tutte le tuple con language diverso da 11.

DFfilteredlanguageitems = DFitems_anobii.filter(DFitems_anobii.language == "11")
#DFfilteredlanguageitems.show(20, False)

#FILTRAGGIO 2: solo "hardcover" e "paperback"

DFfilteredbindingitems = DFfilteredlanguageitems.filter((DFfilteredlanguageitems.binding == "Paperback") | (DFfilteredlanguageitems.binding == "Hardcover"))
#DFfilteredbindingitems.show(20, False)

#FILTRAGGIO 3: eliminare i fumetti (per quanto possibile)
#Userò come parole chiave personaggi famosi Disney quali Topolino e Paperino e i fumetti Bonelli (che fra gli italiani
#sono probabilmente i più gettonati)

DFfilteredmagazineitems = DFfilteredbindingitems.filter(~(DFfilteredbindingitems.title.contains("Topolino")) | (DFfilteredbindingitems.title.contains("Paperino"))|(DFfilteredbindingitems.title.contains("Tex"))|(DFfilteredbindingitems.title.contains("Dylan Dog"))|(DFfilteredbindingitems.title.contains("Nathan Never"))|(DFfilteredbindingitems.title.contains("Zagor")))
#DFfilteredmagazineitems.show(20, False)

#FILTRAGGIO 4: libri con più di X votes. Per vedere i voti di ogni libro dovrei usare il dataframe link_person_item.csv.
#Come possiamo notare, alcune recensione hanno valore 0: questo non vuol dire che il libro è stato valutato
#atrocemente ma che l'utente ha letto il libro senza inserire voti.

#Dobbiamo dunque scremare tali voti da questo dataframe.

file = "/data/SMARTDATA/books/anobii/link_person_item.csv" #MOSTRA IL VOTO DI UN UTENTE A UN LIBRO
DFstars = spark.read.csv(file,header=True)
#DFstars.show(50, False)

DFstarsfilteredno0 = DFstars.filter(DFstars.item_review > 0)
#DFstarsfilteredno0.show(20,False)

#Contiamo i libri con più voti e visualizziamoli in ordine discendente

DFgrouped = DFstarsfilteredno0.groupby("item_id").count().withColumnRenamed("count","total_votes").sort("total_votes", ascending=False)
#I voti dei libri più votati si aggirano intorno ai 10^4 come ordine di grandezza per cui per ora scelgo 300 come
#soglia di voti necessaria a rimanere nel dataframe (soggetto a cambiamenti)
DFgroupedfiltered = DFgrouped.filter(DFgrouped['total_votes'] > 300)
DFjoinbookstars = DFfilteredmagazineitems.join(DFgroupedfiltered, DFfilteredmagazineitems.item_id == DFgroupedfiltered.item_id).drop(DFgroupedfiltered.item_id)

#FILTRAGGIO 5: Pulizie varie ed eventuali
#DFfilteredlessthan10items.filter(DFfilteredlessthan10items.total_review > 100).show()

#NOTA: prendo i titoli con le minuscole per facilità di merging con le biblioteche
DFfiltered = DFjoinbookstars.select("item_id", lower(DFjoinbookstars.title), lower(DFjoinbookstars.sub_title), "isbn", "average_rating", "total_review", "total_wishlist", DFjoinbookstars['total_votes']).withColumnRenamed("lower(title)", "title").withColumnRenamed("lower(sub_title)", "sub_title")
#DFfiltered.sort("total_votes", ascending=False).show(20, False)
#DFfiltered.count() #2516 books (maybe too few? Let's try with 300 WITH 300: 4485, seems fair)

In [15]:
#Let's join votes and books to get titles (not considering votes with 0)

DFvoteswithtitles = DFstarsfilteredno0.join(DFfiltered, DFfiltered.item_id == DFstarsfilteredno0.item_id).select(DFstarsfilteredno0.item_id, "person_id", "title", "sub_title", "item_review", "total_review", "average_rating", "total_votes", "isbn")
#DFvoteswithtitles.show(20, False)

In [16]:
#from lightfm import LightFM
from pyspark.sql.functions import row_number, lit, dense_rank
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
#Per ora proverò a costruire una CSR (matrice sparsa compressa) dal dataset di anobii (senza join con le biblioteche,
#solo per testare). La CSR è costruita in modo tale da avere tre vettori numpy, uno per i dati e due per gli indici
#riga-colonna con cui poi ricostruire la matrice originale.

#Come posso passare dal dataset a questa matrice? La maniera più efficiente (ed è meglio adottarle dato le grandezze
#dei dataset) è quella di creare la matrice direttamente, senza prima crearne una vuota da riempire.

#Supposto che le righe di tale matrice siano gli utenti e le colonne siano i libri (e i dati siano per ora solo
#i voti, poi potremo aggiungere feature per i metadati) allora posso provare ad aggiungere al mio dataframe delle 
#colonne indicanti gli indici futuri di riga colonna, usando degli id incrementali.

#Proviamo. (Forse con gli RDD è meglio)
RDDvoteswithtitles = DFvoteswithtitles.rdd.cache()
#RDDpair = RDDvoteswithtitles.map(lambda x: (x.person_id, list(x))).sortByKey()#.collect()

def create_user_dictionary(rdd): #Used to assign an integer id to each user of the rdd (to get the rows of the CSR)
    rdd = rdd.map(lambda x: (x.person_id, list(x))).sortByKey()
    user_dictionary = rdd.countByKey()
    i = 0
    for key in user_dictionary.keys():
        user_dictionary[key] = i
        i += 1
    return user_dictionary

def create_book_dictionary(rdd): #Used to assign an integer id to each book of the rdd (to get the column of the CSR)
    rdd = rdd.map(lambda x: (x.item_id, list(x))).sortByKey()
    book_dictionary = rdd.countByKey()
    i = 0
    for key in book_dictionary.keys():
        book_dictionary[key] = i
        i += 1
    return book_dictionary

user_dictionary = create_user_dictionary(RDDvoteswithtitles)
book_dictionary = create_book_dictionary(RDDvoteswithtitles)

def addRowColumnId(line): #Funzione usata per aggiungere gli indici per ogni libro e utente, inserisco anche "anobii" per specificare il tipo di provenienza del dato (potrebbe servire per il merge)
    book_number = book_dictionary[line.item_id] 
    user_number = user_dictionary[line.person_id]
    
    return Row(line.item_id, line.person_id, line.title, str(line.sub_title), line.item_review, line.total_review, line.average_rating, str(line.total_votes), line.isbn, str(user_number), str(book_number), "anobii")
  
RDDmapped = RDDvoteswithtitles.map(addRowColumnId)

#print(RDDmapped.take(20))

DFwithindexes = RDDmapped.toDF(["item_id", "person_id", "title", "sub_title",  "item_review", "total_review", "average_rating", "total_votes", "isbn", "user_index", "book_index", "data_type"]) #Per evitare il problema del sampling in sub_title
#DFwithindexes.show(20, False)

#DFwithindexes contiene i voti di ogni utente dato il libro, altre info utili e gli indici di riga e di colonna
#della eventuale compressed sparse matrix per una facile ed efficiente creazione (per i sistemi di raccomandazione)


In [17]:
#Fatto ciò, riprendiamo il database delle biblioteche filtrato (come si importa un altro file .ipynb?)

%run cleanedData.ipynb #Così a quanto pare

Togli oggetti inutili: Prima 290125  - Dopo 257054
Togli libri in lingua straniera: Prima 290125  - Dopo 228059
Togli Fumetti: Prima 290125  - Dopo 227662
prefiltrati: 210633 - filtrati: 17702
prefiltrati: 210633 - filtrati: 17929
prefiltrati: 998403 - filtrati: 215446
prefiltrati: 5484078 - filtrati: 2154533
i libri sono: 17929


In [20]:
#Per adesso, concentriamoci su DFloand_definitive (i prestiti scremati).
DFauthorsandbooks = DFdisplay.join(DFauthorbooks, DFdisplay.author_id == DFauthorbooks.author_id).drop(DFdisplay.author_id)
DFwithindexes_and_authors = DFwithindexes.join(DFauthorsandbooks, DFwithindexes.item_id == DFauthorsandbooks.item_id).select(DFwithindexes.item_id, "person_id", "title", "sub_title", "author_name", "item_review", "total_review", "average_rating", "total_votes", "isbn", "user_index", "book_index", "data_type")
#Prendo i prestiti (che alla fine, è ciò che mi interessa) con i titoli che, per una formattazione più agevole, rendo minuscoli.
DFloans_with_titles = DFloans_definitive.join(DFmanifestations_definitive, DFloans_definitive.manifestation_id_new == DFmanifestations_definitive.manifestation_id_new).select(DFloans_definitive.manifestation_id_new, "patron_id_md5", "author", lower(DFmanifestations_definitive.title)).withColumnRenamed("lower(title)", "title")

#Di cosa ho bisogno in loans_definitive per renderlo "simile" alle tuple di anobii? Certamente avrò bisogno del titolo, poi del manifestation_id (item_id), del cliente della biblioteca
#(orrispettivo di person_id), di assegnare una item_review provvisoria (abbiamo stabilito, almeno per ora, 4 NOTA: l'average rating lo setto a NULL nelle tuple o lo ricalcolo considerando le 
#biblioteche? Da vedere), di settare le total_review (per ora a NULL?), i total_votes, l'isbn e lo user e book index (a NULL?)

#Il problema di molti campi che lascerei a NULL è che in alcuni casi i titoli non sono esattamente gli stessi a causa della formattazione dei database delle biblioteche. Per stemperare il problema,
#proverò a scindere titolo e sottotitoli in due campi differenti (alla maniera di anobii) e a pulire eventuali simboli strani (ad esempio, i numeri all'inizio di alcuni titoli).

RDDloans_with_titles = DFloans_with_titles.rdd

def title_and_subtitle(line): #Usata per splittare titolo e sottotitolo nel database delle biblioteche e renderli più simili ad anobii
    if len(line.title.split(" : ")) > 1: #Il libro possiede un sottotitolo
        title = line.title.split(" : ")[0]
        sub_title = line.title.split(" : ")[1]
        
        return Row(line.manifestation_id_new, line.patron_id_md5, title, sub_title, line.author, 4, "NULL", "NULL", "NULL", "NULL", "NULL", "NULL", "biblioteche") #NE APPROFITTO ANCHE PER INSERIRE VOTO E CAMPI PER IL MERGE (PER ORA NULL)
    else: #Il libro non possiede un sottotitolo
        return Row(line.manifestation_id_new, line.patron_id_md5, line.title, "NULL", line.author, 4, "NULL", "NULL", "NULL", "NULL", "NULL", "NULL", "biblioteche")
    
RDDloans_for_merge = RDDloans_with_titles.map(title_and_subtitle)  

DFloans_to_merge = RDDloans_for_merge.toDF(["item_id", "person_id", "title", "author_name", "sub_title", "item_review", "total_review", "average_rating", "total_votes", "isbn", "user_index", "book_index", "data_type"])

from pyspark.sql.functions import rand

DFmerged = DFwithindexes_and_authors.union(DFloans_to_merge)



In [21]:
DFmerged.orderBy(rand()).show(200, False)

+-------+--------------------------------+-----------------------------------------------------+-------------------------------------------------+------------------------------------+-----------+------------+----------------+-----------+----------+----------+----------+-----------+
|item_id|person_id                       |title                                                |sub_title                                        |author_name                         |item_review|total_review|average_rating  |total_votes|isbn      |user_index|book_index|data_type  |
+-------+--------------------------------+-----------------------------------------------------+-------------------------------------------------+------------------------------------+-----------+------------+----------------+-----------+----------+----------+----------+-----------+
|1677736|777681                          |il mondo nuovo                                       |None                                             |Aldou